In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Qubits messen

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Paketversionen</b></summary>

Der Code auf dieser Seite wurde mit den folgenden Anforderungen entwickelt.
Wir empfehlen die Nutzung dieser oder neuerer Versionen.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>
Um Informationen über den Zustand eines Qubits zu erhalten, kannst du es auf ein [klassisches Bit](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.Clbit) _messen_ (_measure_). In Qiskit werden Messungen in der Berechnungsbasis durchgeführt, das heißt, der Ein-Qubit-Pauli-$Z$-Basis. Daher liefert eine Messung 0 oder 1, abhängig von der Überlappung mit den Pauli-$Z$-Eigenzuständen $|0\rangle$ und $|1\rangle$:

$$
|q\rangle \xrightarrow{measure}\begin{cases}
      0 (\text{outcome}+1), \text{with probability } p_0=|\langle q|0\rangle|^{2}\text{,} \\
      1 (\text{outcome}-1), \text{with probability } p_1=|\langle q|1\rangle|^{2}\text{.}
    \end{cases}
$$

## Mid-Circuit-Messungen
Mid-Circuit-Messungen sind eine Schlüsselkomponente von dynamischen Circuits. Vor `qiskit-ibm-runtime` v0.43.0 war `measure` die einzige Messanweisung in Qiskit. Mid-Circuit-Messungen haben jedoch andere Tuning-Anforderungen als _terminale_ Messungen (Messungen, die am Ende eines Circuits stattfinden). Zum Beispiel musst du die Dauer der Anweisung beim Tuning einer Mid-Circuit-Messung berücksichtigen, da längere Anweisungen verrauschtere Circuits verursachen. Du musst die Anweisungsdauer für terminale Messungen nicht berücksichtigen, da es nach terminalen Messungen keine Anweisungen mehr gibt.

In `qiskit-ibm-runtime` v0.43.0 wurde die Anweisung `MidCircuitMeasure` eingeführt. Wie der Name schon sagt, ist es eine neue Messanweisung, die für die Mid-Circuit-Ausführung auf IBM&reg; QPUs optimiert ist.

> **Note:** Die Anweisung `MidCircuitMeasure` wird auf die Anweisung `measure_2` abgebildet, die in den `supported_instructions` des Backends gemeldet wird. Jedoch wird `measure_2` nicht auf allen Backends unterstützt. Nutze `service.backends(filters=lambda b: "measure_2" in b.supported_instructions)`, um Backends zu finden, die sie unterstützen. Neue Messungen könnten in der Zukunft hinzugefügt werden, aber das ist nicht garantiert.

## Eine Messung auf einen Circuit anwenden
Es gibt mehrere Möglichkeiten, Messungen auf einen Circuit anzuwenden:

### `QuantumCircuit.measure` Methode
Nutze die Methode [`measure`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#measure), um einen [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#quantumcircuit-class) zu messen.

Beispiele:

In [1]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(5, 5)
qc.x(0)
qc.x(1)
qc.x(4)
qc.measure(
    range(5), range(5)
)  #  Measures all qubits into the corresponding clbit.

In [2]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(3, 1)
qc.x([0, 2])
qc.measure(1, 0)  # Measure qubit 1 into the classical bit 0.

### `Measure` class

The Qiskit [Measure](/docs/api/qiskit/circuit#qiskit.circuit.Measure) class measures the specified qubits.

In [3]:
from qiskit.circuit import Measure

qc = QuantumCircuit(3, 1)
qc.x([0, 1])
qc.append(Measure(), [0], [0])  # measure qubit 0 into clbit 0

### `QuantumCircuit.measure_all` method

To measure all qubits into the corresponding classical bits, use the [`measure_all`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#measure_all) method. By default, this method adds new classical bits in a `ClassicalRegister` to store these measurements.

In [4]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(3, 1)
qc.x([0, 2])
qc.measure_all()  # Measure all qubits.

### `Measure` Klasse
Die Qiskit Klasse [Measure](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.Measure) misst die spezifizierten Qubits.

In [5]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(3, 1)
qc.x([0, 2])
qc.measure_active()  # Measure qubits that are not idle, that is, qubits 0 and 2.

<span id="midcircuit"></span>
### `MidCircuitMeasure` method


Use `MidCircuitMeasure` to apply a mid-circuit measurement (requires `qiskit-ibm-runtime` v0.43.0 or later).  While you can use `QuantumCircuit.measure` for a mid-circuit measurement, because of its design, `MidCircuitMeasure` is typically a better choice.  For example, it adds less overhead to your circuit than when using `QuantumCircuit.measure`.

In [6]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.circuit import MidCircuitMeasure
from qiskit.circuit import Measure

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

circ = QuantumCircuit(2, 2)
circ.x(0)
circ.append(MidCircuitMeasure(), [0], [0])
# circ.measure([0], [0])
# circ.measure_all()
print(circ.draw(cregbundle=False))

     ┌───┐┌────────────┐
q_0: ┤ X ├┤0           ├
     └───┘│            │
q_1: ─────┤  Measure_2 ├
          │            │
c_0: ═════╡0           ╞
          └────────────┘
c_1: ═══════════════════
                        


### `QuantumCircuit.measure_all` Methode
Um alle Qubits in die entsprechenden klassischen Bits zu messen, nutze die Methode [`measure_all`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#measure_all). Standardmäßig fügt diese Methode neue klassische Bits in einem `ClassicalRegister` hinzu, um diese Messungen zu speichern.